In [2]:
# Install necessary packages (Kaggle-ready)
!pip install --no-cache-dir transformers==4.43.3 accelerate timm onnxruntime
!pip install --no-cache-dir protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [4]:
# =========================
# 1) Install and imports
# =========================
!pip -q install --no-cache-dir transformers==4.43.3 accelerate==0.33.0 timm==1.0.9 onnxruntime==1.18.1

import os, cv2, glob, math, random, numpy as np, pandas as pd
from pathlib import Path
import torch
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation

In [5]:
# -------------------------
# 2) Paths and parameters
# -------------------------
# Adjust INPUT_DIR to your Kaggle dataset of clean face images
INPUT_DIR  = "/kaggle/input/human-faces/Humans"     # directory with clean .jpg/.png face images
OUTPUT_DIR = "/kaggle/working/synthetic_scars_faceparsing"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/scarred", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/masks", exist_ok=True)            # lesion masks (binary)
os.makedirs(f"{OUTPUT_DIR}/face_skin", exist_ok=True)        # face-skin masks (binary)
os.makedirs(f"{OUTPUT_DIR}/edges_clean", exist_ok=True)      # Canny edges from clean images
os.makedirs(f"{OUTPUT_DIR}/edges_scarred", exist_ok=True)    # optional: edges from scarred

SAVE_CSV = f"{OUTPUT_DIR}/dataset_manifest.csv"
TARGET_SIZE = 512     # standard SD/ControlNet size
SEED = 42
random.seed(SEED); np.random.seed(SEED)

# Limit number of processed images (set to None to process all)
N_SAMPLES = 100

In [6]:
# -------------------------
# 3) Load face-parsing model
# -------------------------
# Using a public face-parsing model hosted on Hugging Face; if internet is restricted,
# upload the model as a Kaggle Dataset and point model_id to that path.
# The model should output a per-pixel class map (CelebAMask-HQ-style 19-part parsing).
model_id = "jonathandinu/face-parsing"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoImageProcessor.from_pretrained(model_id)
model = AutoModelForSemanticSegmentation.from_pretrained(model_id).to(device).eval()

preprocessor_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/339M [00:00<?, ?B/s]

In [7]:
# -------------------------
# 4) Helpers: resize, edges
# -------------------------
def load_and_fit(img_path, size=512):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None: return None
    h, w = img.shape[:2]
    scale = size / max(h, w)
    img = cv2.resize(img, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)
    # pad to square
    h, w = img.shape[:2]
    top = (size - h)//2; bottom = size - h - top
    left = (size - w)//2; right = size - w - left
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REFLECT_101)
    return img

def canny_edges(img_bgr, lo=80, hi=180):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, lo, hi)
    return edges

In [10]:
# -------------------------
# 5) Face parsing → skin mask
# -------------------------
# NOTE: Class indices may differ by model; the code prints a histogram once to help verify.
# Typical CelebAMask-HQ conventions include separate labels for skin, hair, eyes, and mouth.
# Adjust the CLASS_TO_KEEP/EXCLUDE mapping after inspecting printed histograms.

PRINT_HIST_ONCE = True

def parse_face_parts(img_bgr):
    # model expects RGB
    rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    inputs = processor(images=rgb, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**inputs)
        logits = out.logits  # [1, num_classes, H', W']
        upsampled = torch.nn.functional.interpolate(
            logits,
            size=rgb.shape[:2],
            mode="bilinear",
            align_corners=False
        )
        parsing = upsampled.argmax(dim=1).squeeze(0).detach().cpu().numpy().astype(np.int32)
    return parsing

# Example mapping (verify ids via histogram output)
# Adjust these after seeing the first printed histogram for your model.
CLASS_KEEP_AS_SKIN = set([1, 2, 7, 8, 10])  # skin, neck/ear/nose-like regions (example; verify)
CLASS_EXCLUDE = set([17, 4, 5, 11, 12, 13]) # hair and eyes/mouth (example; verify)

def parsing_to_skin_mask(parsing):
    skin = np.zeros_like(parsing, dtype=np.uint8)
    for cid in CLASS_KEEP_AS_SKIN:
        skin[parsing == cid] = 255
    for cid in CLASS_EXCLUDE:
        skin[parsing == cid] = 0
    # Clean up: remove small speckles and soften edges
    k = np.ones((3,3), np.uint8)
    skin = cv2.morphologyEx(skin, cv2.MORPH_OPEN, k, iterations=1)
    skin = cv2.GaussianBlur(skin, (7,7), 0)
    _, skin = cv2.threshold(skin, 127, 255, cv2.THRESH_BINARY)
    return skin